# INTRO

In [51]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', None)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# IMPORT CSV _ lista de doutores

In [52]:
df = pd.read_csv('doutoresFeaturesParte_1.csv', encoding='UTF-8', nrows=200) #, dtype=str, nrows=1000)

df = df.fillna(' ')

df = df.astype({
    'IDLattes': int,
    'DuracaoDoutorado': float,
    'Genero': str,
    'GrandesAreas': str,
    'ExclusividadeDoutorado': str,
    'Publicacoes': float,
    'IdadeAcademica': str,
    'ListaFiliacoes': str,
    'UltimaFiliacao': str,
})

df_doutores = df.copy(deep=True)
df_doutores

# Nova Coluna com Resultado
df_doutores['HistoricoFiliacoesEmIESInternacional'] = ''
df_doutores['UltimaFiliacaoEmIES'] = ''

# IMPORT CSV _ IES Brasil

In [53]:
# LISTA DE IES NACIONAIS
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

df_uni_br = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_br = df_uni_br[['NOME_DA_IES', 'SIGLA']]
df_uni_br = df_uni_br.fillna('')
df_uni_br = df_uni_br.sort_values(['NOME_DA_IES'], ascending=False)
df_uni_br

,NOME_DA_IES,SIGLA
862,iCEV - Instituto de Ensino Superior,
2205,VOAS - INSTITUTO DE APRENDIZAGEM SUPERIOR,VOAS
2873,Universidade do Estado de Mato Grosso Carlos A...,UNEMAT
...,...,...
30,ACADEMIA DE POLÍCIA MILITAR DE MINAS GERAIS,APM
3933,ACADEMIA DA FORÇA AÉREA,AFA
2106,ABEU - CENTRO UNIVERSITÁRIO,UNIABEU


# IMPORT CSV _ IES Mundial

In [54]:
# Lista de IES Mundial
# https://zenodo.org/records/13965926
# https://zenodo.org/records/13965926

df_uni_world = pd.read_csv('ror.csv', encoding='UTF-8')

df_uni_world = df_uni_world[['name', 'aliases', 'acronyms', 'country.country_code']]
df_uni_world = df_uni_world.rename(columns={
    'country.country_code': 'PAIS',
    'name': 'NOME_DA_IES',
    'acronyms': 'SIGLA',
    'aliases': 'ALIASES'
})

# IES BR foram removidos pois os nomes não estão em inglês e assim ficaria duplicado em relação a dataset do MEC
df_uni_world = df_uni_world[df_uni_world['PAIS'] != 'BR']

df_uni_world = df_uni_world.fillna('')

df_uni_world

,NOME_DA_IES,ALIASES,SIGLA,PAIS
0,RMIT University,Royal Melbourne Institute of Technology Univer...,RMIT,AU
1,La Trobe University,,,AU
2,Central Queensland University,CQUniversity,CQU,AU
...,...,...,...,...
111065,Universidad a Distancia de Madrid,Udima,UDIMA,ES
111066,Université du Québec à Rimouski,,UQAR,CA
111067,Argonne National Laboratory,United States Department of Energy Office of S...,ANL,US


# PRE-PROCESSAMENTO

In [55]:
def clean_names(df, column_name, remove_semicolon=True):

    # Remover '&'
    df[column_name] = df[column_name].replace('&AMP;', ' ', regex=True)

    # Remover todos os caracteres nao alfa-numericos, exceto whitespace e ;
    if remove_semicolon:
        df[column_name] = df[column_name].str.replace('[^\w\s]', ' ', regex=True)
    else:
        df[column_name] = df[column_name].str.replace('[^\w\s;]', ' ', regex=True)

    # Normalizar a string
    df[column_name] = df[column_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Remover stopwords
    stop = ['DO', 'DA', 'DE', 'E']
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Strip whitespace
    df[column_name] = df[column_name].apply(lambda x: x.strip())

    return df

In [56]:
df_doutores = df_doutores.apply(lambda x:x.astype(str).str.upper())
df_doutores = clean_names(df_doutores, 'ListaFiliacoes', False)
df_doutores = clean_names(df_doutores, 'UltimaFiliacao', False)

df_uni_br = df_uni_br.apply(lambda x:x.astype(str).str.upper())
df_uni_br = clean_names(df_uni_br, 'NOME_DA_IES')

In [57]:
# Split lista de IES por inicio do nome
df_uni_br.sort_values(by=["NOME_DA_IES"], ascending=True)

df_uni_outros = df_uni_br.copy(deep=True)

df_uni_instituto = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')].index, inplace=True)

df_uni_faculdades = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')].index, inplace=True)

df_uni_escola = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')].index, inplace=True)

df_uni_universidade = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')].index, inplace=True)

df_uni_instituicao = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')].index, inplace=True)

# List
# df_uni_outros, df_uni_instituto, df_uni_faculdades, df_uni_escola, df_uni_universidade, df_uni_instituicao

# Merge IES df

In [58]:
df_uni_br['PAIS'] = 'BR'
df_uni_br['ALIASES'] = ''

print( (df_uni_world.shape[0] + df_uni_br.shape[0]), ' = ', df_uni_world.shape[0], ' + ', df_uni_br.shape[0])

df_uni = pd.concat([df_uni_br, df_uni_world])
df_uni

113552  =  109224  +  4328


,NOME_DA_IES,SIGLA,PAIS,ALIASES
862,ICEV INSTITUTO ENSINO SUPERIOR,,BR,
2205,VOAS INSTITUTO APRENDIZAGEM SUPERIOR,VOAS,BR,
2873,UNIVERSIDADE ESTADO MATO GROSSO CARLOS ALBERTO...,UNEMAT,BR,
...,...,...,...,...
111065,Universidad a Distancia de Madrid,UDIMA,ES,Udima
111066,Université du Québec à Rimouski,UQAR,CA,
111067,Argonne National Laboratory,ANL,US,United States Department of Energy Office of S...


# PLN

## TF_IDF

In [59]:
import re

def get_uni_list_df(nome_instituicao):
    if 'INSTITUTO' in nome_instituicao:
        return df_uni_instituto, tfidf_matrix_instituto
    elif 'FACULDADE' in nome_instituicao:
        return df_uni_faculdades, tfidf_matrix_faculdades
    elif 'ESCOLA' in nome_instituicao:
        return df_uni_escola, tfidf_matrix_escola
    elif 'UNIVERSIDADE' in nome_instituicao:
        return df_uni_universidade, tfidf_matrix_universidade
    elif 'INSTITUICAO' in nome_instituicao:
        return df_uni_instituicao, tfidf_matrix_instituicao

    return df_uni_outros, tfidf_matrix_outros


def iterate_sigla(nome_instituicao, df_for_search):

    lista = df_for_search["SIGLA"].apply(lambda x: True if x in nome_instituicao else False)
    as_coisa = df_for_search[lista]

    if as_coisa.empty:
        return ''
    else:
        return as_coisa['SIGLA'][0], as_coisa['PAIS'][0]

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vetorização TF-IDF : Lista IES Brasil
vectorizer_b = TfidfVectorizer().fit(df_uni_br['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer_b.transform(df_uni_br['NOME_DA_IES']) # geral
tfidf_matrix_instituto = vectorizer_b.transform(df_uni_instituto['NOME_DA_IES'])
tfidf_matrix_faculdades = vectorizer_b.transform(df_uni_faculdades['NOME_DA_IES'])
tfidf_matrix_escola = vectorizer_b.transform(df_uni_escola['NOME_DA_IES'])
tfidf_matrix_universidade = vectorizer_b.transform(df_uni_universidade['NOME_DA_IES'])
tfidf_matrix_instituicao = vectorizer_b.transform(df_uni_instituicao['NOME_DA_IES'])
tfidf_matrix_outros = vectorizer_b.transform(df_uni_outros['NOME_DA_IES'])

# Vetorização TF-IDF : Lista IES Mundial
vectorizer_w = TfidfVectorizer().fit(df_uni_world['NOME_DA_IES'])
tfidf_matrix_world = vectorizer_w.transform(df_uni_world['NOME_DA_IES'])

## UltimaFiliacaoEmIES

In [70]:
threshold = 0.8

def metodo1(nomes_instituicoes):

    if nomes_instituicoes:

        nomes_instituicoes_split = str(nomes_instituicoes).split(';')

        for nome_instituicao in nomes_instituicoes_split:

            nome_instituicao = nome_instituicao.strip()

            print(nome_instituicao)

            # Ver se contém a sigla
            sigla_detectado, _ = iterate_sigla(nome_instituicao, df_uni)
            if sigla_detectado:
                print("sigla detectado : ", sigla_detectado)
                print("==================")
                return '1'

            # Se nao contem a sigla, procurar por nome completo
            else:
                vector_comparacao = vectorizer_b.transform([nome_instituicao])

                uni_list_df, tfidf_matrix_df = get_uni_list_df(nome_instituicao)

                similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                max_value = np.max(similaridade)
                max_index = np.argmax(similaridade)

                nome_detectado = uni_list_df.iloc[max_index]['NOME_DA_IES']

                if (max_value >= threshold):
                    print("nome detectado : ", nome_detectado, max_value)
                    print("==================")
                    return '1'

                # se nao achou nenhuma ies br, procurar ies estrangeira
                else:
                    vector_comparacao = vectorizer_w.transform([nome_instituicao])
                    uni_list_df, tfidf_matrix_df = df_uni_world, tfidf_matrix_world
                    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                    max_value = np.max(similaridade)
                    max_index = np.argmax(similaridade)

                    nome_detectado = uni_list_df.iloc[max_index]['NOME_DA_IES']

                    if (max_value >= threshold):
                        print("nome detectado : ", nome_detectado, max_value)
                        print("==================")
                        return '1'

                    else:
                        print("==================")

                    # se nao passar no threshold, so lamento

        print("==================")
        return '0'

df_doutores['UltimaFiliacaoEmIES'] = 'N'
df_doutores['UltimaFiliacaoEmIES'] = df_doutores["UltimaFiliacao"].apply(lambda x: metodo1(x))

INSTITUTO ASTRONOMIA GEOFISICA CIENCIAS ATMOSFERICAS UNIVERSIDADE
CLEAN WEINHEIM INTERNET
REVISTA ABPI
ASSOCIACAO ACADEMICA PROPRIEDADE INTELECTUAL
CONSELHO NACIONAL DESENVOLVIMENTO CIENTIFICO TECNOLOGICO
LIVROS PROFNIT
CONSELHO NACIONAL DESENVOLVIMENTO CIENTIFICO TECNOLOGICO
CHEMICAL ENGINEERING RESEARCH AMP DESIGN
INDIAN JOURNAL OF PHARMACEUTICAL SCIENCES
UNIVERSIDADE CANDIDO MENDES
nome detectado :  UNIVERSIDADE CANDIDO MENDES 1.0000000000000002
REVISTA BRASILEIRA ENFERMAGEM
PETROLEO BRASILEIRO SA
nome detectado :  SA Health 0.9046827764423087
UNIVERSIDADE FEDERAL DOS VALES JEQUITINHONHA MUCURI CAMPUS JK
nome detectado :  UNIVERSIDADE FEDERAL DOS VALES JEQUITINHONHA MUCURI 0.8511044705553963
AEVA INC
nome detectado :  INC Research (United States) 0.8165881699754524
GEOHECOLABORATORIO GEO HIDROECOLOGIA GESTAO RISCOS
INSTITUTO FEDERAL SUL RIO GRANDENSE
nome detectado :  INSTITUTO FEDERAL EDUCACAO CIENCIA TECNOLOGIA SUL RIO GRANDENSE 0.8541763065396581
ASSOCIACAO CULTURAL EDUCACIONAL F

## HistoricoFiliacoesEmIESInternacional

In [71]:
def metodo1(nomes_instituicoes):

    if nomes_instituicoes:

        nomes_instituicoes_split = str(nomes_instituicoes).split(';')

        for nome_instituicao in nomes_instituicoes_split:

            nome_instituicao = nome_instituicao.strip()

            print(nome_instituicao)

            # Ver se contém a sigla
            sigla_detectado, pais = iterate_sigla(nome_instituicao, df_uni)
            if pais != 'BR':
                print("sigla estrangeira detectado : ", sigla_detectado)
                print("==================")
                return '1'

            # Se nao contem a sigla, procurar por nome completo
            else:
                vector_comparacao = vectorizer_b.transform([nome_instituicao])

                uni_list_df, tfidf_matrix_df = get_uni_list_df(nome_instituicao)

                similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                max_value = np.max(similaridade)
                max_index = np.argmax(similaridade)

                nome_detectado = uni_list_df.iloc[max_index]['NOME_DA_IES']

                if (max_value >= threshold):
                    print("nome detectado : ", nome_detectado, max_value)
                    print("==================")

                # se nao achou nenhuma ies br, procurar ies estrangeira
                else:
                    vector_comparacao = vectorizer_w.transform([nome_instituicao])
                    uni_list_df, tfidf_matrix_df = df_uni_world, tfidf_matrix_world
                    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                    max_value = np.max(similaridade)
                    max_index = np.argmax(similaridade)

                    nome_detectado = uni_list_df.iloc[max_index]['NOME_DA_IES']

                    if (max_value >= threshold):
                        print("nome estrangeiro detectado : ", nome_detectado, max_value)
                        print("==================")
                        return '1'

                    else:
                        print("==================")

                    # se nao passar no threshold, so lamento

        print("==================")
        return '0'

df_doutores['HistoricoFiliacoesEmIESInternacional'] = 'N'
df_doutores['HistoricoFiliacoesEmIESInternacional'] = df_doutores["ListaFiliacoes"].apply(lambda x: metodo1(x))

UNIVERSIDADE FEDERAL ABC
nome detectado :  FUNDACAO UNIVERSIDADE FEDERAL ABC 0.8616346145053622
INSTITUTO FISICA USP SAO CARLOS
INSTITUTO PESQUISAS ENERGETICAS NUCLEARES
IEEE PHOTONICS JOURNAL
APPLIED OPTICS
INSTITUTO ASTRONOMIA GEOFISICA CIENCIAS ATMOSFERICAS UNIVERSIDADE
UNIVERSIDADE ESTADUAL MARINGA
nome detectado :  UNIVERSIDADE ESTADUAL MARINGA 1.0
INDUSTRIAL AMP ENGINEERING CHEMISTRY RESEARCH
UNIVERSIDADE TECNOLOGICA FEDERAL PARANA
nome detectado :  UNIVERSIDADE TECNOLOGICA FEDERAL PARANA 1.0000000000000002
CLEAN WEINHEIM INTERNET
PADETEC PARQUE DESENVOLVIMENTO TECNOLOGICO SC
FACULDADE FILOSOFIA CIENCIAS CETRAS RIBERAO PRETO
VEGETEX MERCK
nome estrangeiro detectado :  Merck (Germany) 0.8930646711709934
UNIVERSIDADE ESTADUAL PAULISTA JULIO MESQUITA FILHO
nome detectado :  UNIVERSIDADE ESTADUAL PAULISTA JULIO MESQUITA FILHO 1.0
UNIVERSIDADE FEDERAL SAO CARLOS
nome detectado :  UNIVERSIDADE FEDERAL SAO CARLOS 1.0000000000000002
INSTITUTO FEDERAL SAO PAULO
CONSELHO NACIONAL DESENVOLV

In [72]:
df_doutores

,IDLattes,DuracaoDoutorado,Genero,GrandesAreas,ExclusividadeDoutorado,Publicacoes,IdadeAcademica,ListaFiliacoes,UltimaFiliacao,HistoricoFiliacoesEmIESInternacional,UltimaFiliacaoEmIES
0,39015885890,4.0,,,,9.0,9.0,UNIVERSIDADE FEDERAL ABC; INSTITUTO FISICA USP...,INSTITUTO ASTRONOMIA GEOFISICA CIENCIAS ATMOSF...,0,0
1,104116232000,4.0,,,,8.0,5.0,UNIVERSIDADE ESTADUAL MARINGA; INDUSTRIAL AMP ...,CLEAN WEINHEIM INTERNET,0,0
2,111305956227,4.0,,,,17.0,15.0,PADETEC PARQUE DESENVOLVIMENTO TECNOLOGICO SC;...,REVISTA ABPI; ASSOCIACAO ACADEMICA PROPRIEDADE...,1,0
...,...,...,...,...,...,...,...,...,...,...,...
197,4515964877163,2.0,,,,49.0,35.0,UNIVERSIDADE FEDERAL RURAL SEMI ARIDO; UNIVERS...,UNIVERSIDADE FEDERAL RURAL SEMI ARIDO,0,1
198,4530949909455,4.0,,,,3.0,29.0,UNIVERSIDADE CAXIAS SUL; INSTITUTO CRIMINALIST...,UNIVERSIDADE FEDERAL SANTA MARIA,1,1
199,4597502232092,2.0,,,,107.0,27.0,UNIVERSIDADE ESTADUAL CAMPINAS; REVISTA FACULD...,ORAL DISEASES 1354 523X,0,0
